In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from src.env.dunderbot_env import DunderBotEnv
import src.run as run
import src.load as load

loading config from config/config.yml
The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



# TODO:


- load interesting data, specified in config
- understand normalization of values
- understand the rendering, especially during learning
- understand the time steps in learn and predict
- setup train/test functionality (separate reset functions? Multiple time periods for testing?)
- add distributions of actions as output during training and predition (latter more important)
- lock down the seeds (and decide in config from a few specific ones)
- setup performance output, what we need to make data driven decision
- Find as long time series of BTCUSD hourly (and daily) data as possible, quite shortright now


# DONE: 
- add config
- modularize rewards
- own actions
- add more detailed actions for buy and sell amounts (still Discrete, I believe)
- wrap running code in function in main.py (to enable more dynamic data loading and train/test)
- own observations
- remove the bias when getting the observations (suggestion in the repo issues)

# Run

In [ ]:
%%time
df = load.load_OHLC_data()
env = run.preprocess(df=df)
model = run.train(env=env, total_timesteps=20000)

/code/src/env/dunderbot_env.py:82: RuntimeWarning: invalid value encountered in double_scalars
  prev_cost + additional_cost) / (self.shares_held + shares_bought)


--------------------------------------
| approxkl           | 0.00027140765 |
| clipfrac           | 0.0           |
| explained_variance | 5.19e-06      |
| fps                | 219           |
| n_updates          | 1             |
| policy_entropy     | 2.1968966     |
| policy_loss        | -0.003087244  |
| serial_timesteps   | 128           |
| time_elapsed       | 3.22e-05      |
| total_timesteps    | 128           |
| value_loss         | 111310090.0   |
--------------------------------------
--------------------------------------
| approxkl           | 0.0007819383  |
| clipfrac           | 0.0           |
| explained_variance | 2.38e-07      |
| fps                | 426           |
| n_updates          | 10            |
| policy_entropy     | 2.1890085     |
| policy_loss        | -0.0061511537 |
| serial_timesteps   | 1280          |
| time_elapsed       | 3.05          |
| total_timesteps    | 1280          |
| value_loss         | 487802080.0   |
-------------------------

In [ ]:
%%time
run.predict(env=env, model=model)